### Computer Vision and Convolutional Neural Networks in Tensorflow

In [1]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

zip_ref = zipfile.ZipFile('pizza_steak.zip')
zip_ref.extractall()
zip_ref.close()

--2024-06-16 12:32:52--  https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.166.251, 172.217.166.27, 172.217.27.187, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.166.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109540975 (104M) [application/zip]
Saving to: ‘pizza_steak.zip’

pizza_steak.zip     100%[===================>] 104.47M  12.7MB/s    in 8.8s    

2024-06-16 12:33:02 (11.8 MB/s) - ‘pizza_steak.zip’ saved [109540975/109540975]

